<a href="https://colab.research.google.com/github/Jin-K-Yang/Clustering-by-Louvain-Algorithm/blob/main/louvain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from glob import glob


# load excel file
files = glob("*.xlsx")
df = pd.concat([pd.read_excel(f, usecols='F, K', converters={'部門':str}) for f in files])
df.reset_index(drop=True)

# convert into two array which represent customer and product
array = df.to_numpy()
split_array = np.hsplit(array, 2)
customer = split_array[0]
product = split_array[1]

# construct bipartite graph
G = nx.Graph()
for i in range(len(product)):
    # zero padding
    product[i][0] = product[i][0].zfill(4)

    # add nodes
    G.add_nodes_from(product[i], bipartite = 0)
    G.add_nodes_from(customer[i], bipartite = 1)
    
    # add edges
    if G.has_edge(product[i][0], customer[i][0]):
        G[product[i][0]][customer[i][0]]['weight'] += 1
    else:
        G.add_edge(product[i][0], customer[i][0], weight = 1)

# show the total number of nodes and edges
# print(G.number_of_nodes())
# print(G.number_of_edges())

# compute the best partition
partition = community_louvain.best_partition(G)

# classify the result
community = {}
for key, value in set(partition.items()):
    if key[0].isalpha():
        community.setdefault(value, {}).setdefault('customer', []).append(key)
    else:
        community.setdefault(value, {}).setdefault('product', []).append(key)

# calculate the sum of edges of clusters
for i in range(len(community)):
    SG = G.subgraph(community[i]['product'] + community[i]['customer'])
    community[i]['edges_sum'] = SG.number_of_edges()

# benefit-cost analysis
benefit = {
    'Customer_Count':[],
    'Product_Count':[],
    'Cost':[],
    'Benefit':[],
    'Ratio':[]
}

for i in range(len(community)):
    benefit['Customer_Count'].append(len(community[i]['customer']))
    benefit['Product_Count'].append(len(community[i]['product']))
    benefit['Cost'].append(len(community[i]['customer']) * len(community[i]['product']))
    benefit['Benefit'].append(community[i]['edges_sum'])
    benefit['Ratio'].append(community[i]['edges_sum'] / (len(community[i]['customer']) * len(community[i]['product'])))

df_benefit = pd.DataFrame(benefit)
df_benefit

,Customer_Count,Product_Count,Cost,Benefit,Ratio
0,2659,6,15954,3733,0.233985
1,1688,11,18568,3069,0.165284
2,1,1,1,1,1.000000
3,939,4,3756,975,0.259585
4,1297,10,12970,1653,0.127448
5,2139,5,10695,2603,0.243385
6,731,6,4386,810,0.184679
7,1044,7,7308,1834,0.250958
8,1478,1,1478,1478,1.000000
9,1185,4,4740,1345,0.283755


In [ ]:
# draw the graph
pos = nx.spring_layout(G)

# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=1, cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()